In [1]:
from selenium import webdriver
driver = webdriver.Chrome('chromedriver.exe')

In [2]:
# 크롤링 사이트 접속하기
driver.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=159&yy=2015&obs=07&x=9&y=18')

In [105]:
# 페이지 크롤링 함수
from bs4 import BeautifulSoup
def crawling():
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')

    content = soup.select('table.table_develop>tbody')

    ex=content[0].text.replace('\n'," ")
    ex = ex.split(' ')
    date = []
    for s in ex:
        if s!='':
            date.append(s)
    return date

In [102]:
def make_list():
    List=[]
    for i in range(13):
        List.append([])
    return List

In [97]:
import pandas as pd

평균기온_2015=make_list()

cnt = 0
for num in date:

    평균기온_2015[cnt%13].append(num)    
    cnt+=1
    
평균기온_2015=평균기온_2015[1:]

평균기온_2015 = pd.DataFrame(평균기온_2015).T
평균기온_2015.drop(31,axis=0,inplace=True)
평균기온_2015_final=[]
for i in range(12):
    for j in range(31):
        평균기온_2015_final.append(평균기온_2015.iloc[j,i])
평균기온_2015_final

In [110]:
date=crawling()
import pandas as pd

평균기온_2016=make_list()
cnt = 0
for num in date:

    평균기온_2016[cnt%13].append(num)    
    cnt+=1
    
평균기온_2016=평균기온_2016[1:]

평균기온_2016 = pd.DataFrame(평균기온_2016).T
평균기온_2016.drop(31,axis=0,inplace=True)
평균기온_2016_final=[]

for i in range(12):
    for j in range(31):
        평균기온_2016_final.append(평균기온_2016.iloc[j,i])
평균기온_2016_final

['5.3',
 '8.1',
 '11.4',
 '10.4',
 '5.3',
 '4.8',
 '3.9',
 '2.5',
 '3.8',
 '4.8',
 '4.5',
 '2.5',
 '1.5',
 '2.9',
 '4.2',
 '5.7',
 '6.6',
 '2.3',
 '-3.1',
 '-2.5',
 '-0.9',
 '0.8',
 '-2.9',
 '-7.6',
 '-2.6',
 '1.4',
 '4.7',
 '4.4',
 '4.6',
 '6.1',
 '3.8',
 '1.7',
 '1.2',
 '1.9',
 '3.2',
 '4.1',
 '2.4',
 '3.3',
 '4.5',
 '6.4',
 '6.3',
 '9.2',
 '12.3',
 '14.5',
 '6.7',
 '-0.4',
 '2.2',
 '3.9',
 '5.7',
 '8.9',
 '7.8',
 '3.8',
 '6.6',
 '7.9',
 '3.0',
 '3.1',
 '6.1',
 '10.6',
 '10.1',
 '1.1',
 '\xa0',
 '\xa0',
 '1.4',
 '6.3',
 '11.4',
 '11.5',
 '13.7',
 '13.3',
 '11.4',
 '9.6',
 '6.9',
 '5.6',
 '3.5',
 '5.1',
 '7.6',
 '7.7',
 '8.2',
 '11.2',
 '12.6',
 '12.7',
 '13.3',
 '10.6',
 '10.2',
 '12.3',
 '11.0',
 '8.6',
 '8.1',
 '7.4',
 '9.5',
 '11.8',
 '13.3',
 '15.0',
 '14.8',
 '16.1',
 '15.1',
 '14.2',
 '11.7',
 '14.3',
 '14.8',
 '12.7',
 '16.5',
 '17.2',
 '15.6',
 '10.5',
 '12.5',
 '14.6',
 '17.4',
 '14.6',
 '15.0',
 '16.5',
 '15.2',
 '14.9',
 '15.4',
 '15.5',
 '16.9',
 '16.7',
 '16.5',
 '16.9',

In [112]:
date=crawling()
import pandas as pd

평균기온_2017=make_list()
cnt = 0
for num in date:

    평균기온_2017[cnt%13].append(num)    
    cnt+=1
    
평균기온_2017=평균기온_2017[1:]

평균기온_2017 = pd.DataFrame(평균기온_2017).T
평균기온_2017.drop(31,axis=0,inplace=True)
평균기온_2017_final=[]

for i in range(12):
    for j in range(31):
        평균기온_2017_final.append(평균기온_2017.iloc[j,i])
평균기온_2017_final

['6.5',
 '9.5',
 '7.9',
 '8.1',
 '7.8',
 '7.5',
 '8.7',
 '9.3',
 '7.3',
 '3.5',
 '1.1',
 '3.9',
 '1.5',
 '-2.4',
 '-1.5',
 '1.4',
 '4.4',
 '5.3',
 '4.9',
 '2.1',
 '0.0',
 '0.6',
 '-2.5',
 '-1.1',
 '1.2',
 '3.8',
 '7.2',
 '6.5',
 '8.0',
 '3.3',
 '3.1',
 '2.3',
 '3.4',
 '5.5',
 '9.5',
 '7.3',
 '4.0',
 '4.4',
 '6.1',
 '1.0',
 '-1.5',
 '-0.5',
 '2.2',
 '4.7',
 '5.5',
 '7.3',
 '11.7',
 '12.1',
 '4.0',
 '6.7',
 '9.4',
 '5.2',
 '8.2',
 '5.4',
 '2.8',
 '5.7',
 '6.4',
 '7.5',
 '8.8',
 '\xa0',
 '\xa0',
 '\xa0',
 '9.3',
 '6.6',
 '6.7',
 '9.7',
 '10.2',
 '7.1',
 '3.1',
 '2.9',
 '6.9',
 '8.6',
 '10.3',
 '11.3',
 '10.9',
 '8.7',
 '7.6',
 '9.0',
 '10.6',
 '11.3',
 '12.4',
 '10.7',
 '9.2',
 '9.1',
 '11.6',
 '9.9',
 '9.1',
 '6.9',
 '7.2',
 '9.3',
 '9.0',
 '12.5',
 '9.9',
 '8.8',
 '9.3',
 '14.9',
 '16.1',
 '14.5',
 '14.7',
 '16.0',
 '16.1',
 '14.3',
 '12.4',
 '12.8',
 '13.7',
 '15.0',
 '15.9',
 '17.8',
 '16.1',
 '15.7',
 '15.2',
 '15.1',
 '15.3',
 '15.5',
 '14.0',
 '14.4',
 '16.1',
 '17.6',
 '14.4',
 '1

In [113]:
date=crawling()
import pandas as pd

평균기온_2018=make_list()
cnt = 0
for num in date:

    평균기온_2018[cnt%13].append(num)    
    cnt+=1
    
평균기온_2018=평균기온_2018[1:]

평균기온_2018 = pd.DataFrame(평균기온_2018).T
평균기온_2018.drop(31,axis=0,inplace=True)
평균기온_2018_final=[]

for i in range(12):
    for j in range(31):
        평균기온_2018_final.append(평균기온_2018.iloc[j,i])
평균기온_2018_final

['2.7',
 '4.1',
 '2.6',
 '2.8',
 '4.1',
 '3.1',
 '5.4',
 '5.8',
 '1.9',
 '-0.7',
 '-4.6',
 '-4.8',
 '-0.2',
 '3.5',
 '9.3',
 '8.8',
 '6.4',
 '7.2',
 '6.0',
 '5.2',
 '6.5',
 '6.1',
 '-1.3',
 '-6.7',
 '-5.1',
 '-5.4',
 '-1.4',
 '1.4',
 '-2.4',
 '-2.1',
 '1.9',
 '3.3',
 '2.6',
 '-0.2',
 '-4.7',
 '-4.4',
 '-5.1',
 '-3.5',
 '0.8',
 '6.1',
 '6.4',
 '-1.1',
 '-0.8',
 '1.8',
 '8.5',
 '7.8',
 '6.6',
 '4.1',
 '4.8',
 '7.0',
 '8.3',
 '5.3',
 '4.2',
 '6.8',
 '8.6',
 '8.6',
 '7.5',
 '9.2',
 '9.6',
 '\xa0',
 '\xa0',
 '\xa0',
 '7.4',
 '4.9',
 '9.6',
 '13.2',
 '6.5',
 '6.4',
 '7.2',
 '4.9',
 '4.8',
 '7.6',
 '9.7',
 '11.2',
 '13.3',
 '15.3',
 '14.3',
 '5.3',
 '6.6',
 '10.7',
 '9.6',
 '5.7',
 '2.9',
 '6.8',
 '9.6',
 '12.4',
 '13.9',
 '15.5',
 '16.0',
 '15.6',
 '15.1',
 '12.7',
 '14.1',
 '16.3',
 '17.9',
 '18.2',
 '11.1',
 '10.5',
 '10.7',
 '6.5',
 '8.8',
 '13.6',
 '14.9',
 '18.7',
 '16.3',
 '13.7',
 '12.5',
 '13.9',
 '12.8',
 '13.9',
 '15.3',
 '15.8',
 '18.8',
 '18.9',
 '17.1',
 '11.8',
 '12.0',
 '13.3'

In [114]:
date=crawling()
import pandas as pd

평균기온_2019=make_list()
cnt = 0
for num in date:

    평균기온_2019[cnt%13].append(num)    
    cnt+=1
    
평균기온_2019=평균기온_2019[1:]

평균기온_2019 = pd.DataFrame(평균기온_2019).T
평균기온_2019.drop(31,axis=0,inplace=True)
평균기온_2019_final=[]

for i in range(12):
    for j in range(31):
        평균기온_2019_final.append(평균기온_2019.iloc[j,i])
평균기온_2019_final

['1.0',
 '1.2',
 '3.0',
 '5.0',
 '5.5',
 '4.4',
 '4.4',
 '3.2',
 '2.3',
 '4.8',
 '6.2',
 '6.2',
 '6.8',
 '7.3',
 '5.6',
 '0.0',
 '2.4',
 '5.5',
 '7.5',
 '5.5',
 '2.8',
 '5.5',
 '6.5',
 '5.7',
 '4.2',
 '1.0',
 '3.9',
 '5.3',
 '5.2',
 '8.6',
 '2.2',
 '0.4',
 '4.2',
 '11.6',
 '6.8',
 '8.0',
 '10.5',
 '8.3',
 '3.2',
 '2.5',
 '4.0',
 '2.9',
 '4.0',
 '4.6',
 '5.2',
 '6.3',
 '3.2',
 '3.2',
 '6.5',
 '7.1',
 '5.6',
 '6.9',
 '9.0',
 '9.2',
 '9.0',
 '10.8',
 '10.9',
 '10.4',
 '10.6',
 '\xa0',
 '\xa0',
 '\xa0',
 '10.6',
 '10.7',
 '11.5',
 '11.4',
 '12.0',
 '10.7',
 '9.7',
 '10.5',
 '11.8',
 '8.4',
 '9.8',
 '9.4',
 '5.5',
 '8.0',
 '8.2',
 '7.7',
 '9.9',
 '11.5',
 '13.9',
 '13.8',
 '15.6',
 '9.4',
 '7.0',
 '6.9',
 '8.7',
 '13.6',
 '14.8',
 '12.8',
 '13.9',
 '11.6',
 '7.7',
 '7.7',
 '9.4',
 '9.8',
 '12.1',
 '13.9',
 '14.4',
 '12.4',
 '13.1',
 '11.2',
 '9.7',
 '10.2',
 '11.8',
 '13.5',
 '13.3',
 '12.7',
 '16.0',
 '18.0',
 '16.4',
 '13.8',
 '14.4',
 '17.2',
 '17.1',
 '16.2',
 '16.8',
 '14.8',
 '11.3',


In [116]:
date=crawling()
import pandas as pd

강수량_2015=make_list()
cnt = 0
for num in date:

    강수량_2015[cnt%13].append(num)    
    cnt+=1
    
강수량_2015=강수량_2015[1:]

강수량_2015 = pd.DataFrame(강수량_2015).T
강수량_2015.drop(31,axis=0,inplace=True)
강수량_2015_final=[]

for i in range(12):
    for j in range(31):
        if 강수량_2015.iloc[j][i]=='\xa0' :
            강수량_2015_final.append(0)
        else:
            강수량_2015_final.append(강수량_2015.iloc[j,i])
강수량_2015_final

[0,
 0,
 0,
 0,
 '0.5',
 '26.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '5.0',
 '3.5',
 0,
 0,
 0,
 0,
 0,
 '3.0',
 '2.5',
 0,
 0,
 '0.1',
 '9.5',
 0,
 0,
 '0.5',
 '0.1',
 0,
 0,
 0,
 0,
 '1.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '2.5',
 '3.5',
 0,
 0,
 0,
 0,
 '13.0',
 '4.5',
 0,
 0,
 0,
 0,
 0,
 '6.0',
 0,
 0,
 0,
 '2.5',
 0,
 '7.5',
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 0,
 '11.0',
 0,
 0,
 0,
 0,
 '45.0',
 '2.5',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '8.0',
 '0.4',
 '4.5',
 '31.0',
 '47.0',
 '5.0',
 '11.5',
 0,
 0,
 0,
 0,
 0,
 '1.5',
 '14.0',
 '12.0',
 0,
 '0.1',
 0,
 '8.0',
 '87.5',
 '8.5',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '2.0',
 '35.0',
 0,
 0,
 0,
 0,
 '42.0',
 0,
 0,
 0,
 0,
 0,
 '0.2',
 0,
 '68.5',
 '37.5',
 0,
 0,
 '0.5',
 '0.3',
 0,
 '30.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '4.0',
 0,
 0,
 '0.1',
 0,
 0,
 '7.0',
 0,
 0,
 '2.0',
 0,
 0,
 '3.0',
 0,
 0,
 0,
 '1.0',
 0,
 0,
 0,
 0,
 '0.5',
 '0.1',
 0,
 0,
 0,
 '17.5',
 '14.5',
 '1.0',
 0,
 0,
 '22.5',
 0,
 '0.1',
 0

In [ ]:
date=crawling()
import pandas as pd

강수량_2015=make_list()
cnt = 0
for num in date:

    강수량_2015[cnt%13].append(num)    
    cnt+=1
    
강수량_2015=강수량_2015[1:]

강수량_2015 = pd.DataFrame(강수량_2015).T
강수량_2015.drop(31,axis=0,inplace=True)
강수량_2015_final=[]

for i in range(12):
    for j in range(31):
        if 강수량_2015.iloc[j][i]=='\xa0' :
            강수량_2015_final.append(0)
        else:
            강수량_2015_final.append(강수량_2015.iloc[j,i])
강수량_2015_final

In [117]:
date=crawling()
import pandas as pd

강수량_2016=make_list()
cnt = 0
for num in date:

    강수량_2016[cnt%13].append(num)    
    cnt+=1
    
강수량_2016=강수량_2016[1:]

강수량_2016 = pd.DataFrame(강수량_2016).T
강수량_2016.drop(31,axis=0,inplace=True)
강수량_2016_final=[]

for i in range(12):
    for j in range(31):
        if 강수량_2016.iloc[j][i]=='\xa0' :
            강수량_2016_final.append(0)
        else:
            강수량_2016_final.append(강수량_2016.iloc[j,i])
강수량_2016_final

[0,
 0,
 0,
 0,
 '0.2',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '14.2',
 0,
 0,
 0,
 0,
 0,
 '0.0',
 '0.0',
 0,
 0,
 0,
 '24.3',
 '20.8',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '32.7',
 '28.9',
 0,
 0,
 0,
 0,
 0,
 0,
 '1.4',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.7',
 '0.0',
 0,
 0,
 0,
 0,
 0,
 '35.7',
 '1.6',
 '32.7',
 0,
 '6.6',
 '13.6',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '38.7',
 0,
 0,
 0,
 0,
 '0.0',
 '0.0',
 '0.0',
 '4.6',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.9',
 '3.8',
 0,
 '18.2',
 '47.4',
 0,
 0,
 0,
 '0.0',
 0,
 '14.8',
 0,
 0,
 '36.6',
 '8.0',
 0,
 0,
 0,
 '31.8',
 0,
 0,
 0,
 0,
 '1.6',
 '32.1',
 '3.3',
 '0.0',
 0,
 0,
 0,
 0,
 '40.7',
 0,
 0,
 '4.7',
 0,
 0,
 '0.0',
 '22.3',
 0,
 0,
 0,
 0,
 '0.0',
 '14.5',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '24.7',
 0,
 '0.0',
 0,
 0,
 '1.9',
 0,
 0,
 0,
 0,
 0,
 '6.6',
 0,
 '3.1',
 0,
 0,
 0,
 0,
 0,
 '8.6',
 0,
 0,
 '0.0',
 '1.5',
 0,
 '2.8',
 '9.1',
 '10.2',
 '3.1',
 '13.4',
 0,
 '53.7',
 0,
 0,
 '0.2',
 '0.0',
 '2.9',
 0,
 0,
 '31.9

In [118]:
date=crawling()
import pandas as pd

강수량_2017=make_list()
cnt = 0
for num in date:

    강수량_2017[cnt%13].append(num)    
    cnt+=1
    
강수량_2017=강수량_2017[1:]

강수량_2017 = pd.DataFrame(강수량_2017).T
강수량_2017.drop(31,axis=0,inplace=True)
강수량_2017_final=[]

for i in range(12):
    for j in range(31):
        if 강수량_2017.iloc[j][i]=='\xa0' :
            강수량_2017_final.append(0)
        else:
            강수량_2017_final.append(강수량_2017.iloc[j,i])
강수량_2017_final

[0,
 0,
 0,
 0,
 '0.7',
 0,
 '0.0',
 '4.6',
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '5.9',
 '0.8',
 0,
 0,
 0,
 0,
 '0.2',
 '4.9',
 0,
 0,
 '0.0',
 '0.0',
 0,
 0,
 0,
 '0.0',
 '0.0',
 0,
 '1.4',
 '2.4',
 0,
 0,
 '1.4',
 0,
 '23.2',
 0,
 0,
 0,
 0,
 '0.3',
 0,
 0,
 0,
 0,
 '13.2',
 '1.6',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 '6.3',
 '2.5',
 0,
 0,
 0,
 '3.3',
 '3.9',
 '2.8',
 0,
 '2.1',
 0,
 '0.0',
 '3.0',
 0,
 0,
 0,
 '10.6',
 '26.6',
 0,
 0,
 0,
 '0.0',
 '15.5',
 0,
 0,
 0,
 '0.0',
 0,
 '37.7',
 0,
 0,
 '0.0',
 0,
 '0.0',
 0,
 0,
 '2.3',
 '9.4',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.1',
 '6.6',
 0,
 0,
 0,
 '20.6',
 '0.1',
 0,
 '11.8',
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 '0.5',
 '18.3',
 0,
 0,
 0,
 '0.0',
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 '9.4',
 0,
 0,
 0,
 '0.1',
 0,
 '0.0',
 0,
 0,
 '21.1',
 '0.4',
 0,
 0,
 '2.3',
 '0.0',
 '122.

In [119]:
date=crawling()
import pandas as pd

강수량_2018=make_list()
cnt = 0
for num in date:

    강수량_2018[cnt%13].append(num)    
    cnt+=1
    
강수량_2018=강수량_2018[1:]

강수량_2018 = pd.DataFrame(강수량_2018).T
강수량_2018.drop(31,axis=0,inplace=True)
강수량_2018_final=[]

for i in range(12):
    for j in range(31):
        if 강수량_2018.iloc[j][i]=='\xa0' :
            강수량_2018_final.append(0)
        else:
            강수량_2018_final.append(강수량_2018.iloc[j,i])
강수량_2018_final

[0,
 0,
 0,
 0,
 '0.8',
 0,
 '0.1',
 '6.4',
 0,
 '0.2',
 0,
 0,
 0,
 0,
 '0.1',
 '27.6',
 '5.4',
 0,
 0,
 0,
 0,
 '0.3',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.2',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 '52.3',
 0,
 0,
 0,
 0,
 0,
 0,
 '39.1',
 '7.7',
 0,
 '3.8',
 '23.5',
 0,
 0,
 0,
 0,
 0,
 0,
 '62.0',
 '2.5',
 0,
 '5.0',
 '37.1',
 '3.4',
 '22.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '5.9',
 '0.0',
 '21.0',
 0,
 0,
 0,
 0,
 '0.2',
 0,
 0,
 '34.1',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 '34.8',
 '60.4',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 '17.0',
 '0.0',
 0,
 0,
 '42.0',
 '30.7',
 0,
 0,
 0,
 0,
 '2.6',
 '18.4',
 0,
 0,
 '3.8',
 '6.6',
 '1.9',
 0,
 0,
 0,
 '4.1',
 '27.6',
 0,
 0,
 0,
 0,
 '0.6',
 0,
 '0.5',
 0,
 0,
 0,
 0,
 0,
 '5.7',
 0,
 0,
 0,
 0,
 '5.2',
 '32.9',
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 '6.8',
 '0.5',
 0,
 0,
 '0.9',
 0,
 0,
 '1.5',
 '6.7',
 '131.1',
 '11.0',
 '74.4',
 0,
 '13.9',
 '2.7'

In [120]:
date=crawling()
import pandas as pd

강수량_2019=make_list()
cnt = 0
for num in date:

    강수량_2019[cnt%13].append(num)    
    cnt+=1
    
강수량_2019=강수량_2019[1:]

강수량_2019 = pd.DataFrame(강수량_2019).T
강수량_2019.drop(31,axis=0,inplace=True)
강수량_2019_final=[]

for i in range(12):
    for j in range(31):
        if 강수량_2019.iloc[j][i]=='\xa0' :
            강수량_2019_final.append(0)
        else:
            강수량_2019_final.append(강수량_2019.iloc[j,i])
강수량_2019_final

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.7',
 0,
 0,
 0,
 0,
 0,
 0,
 '0.5',
 '0.2',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '11.3',
 0,
 0,
 '21.4',
 0,
 0,
 0,
 '0.0',
 0,
 0,
 '0.0',
 0,
 0,
 0,
 0,
 '1.3',
 0,
 0,
 '0.0',
 '28.8',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 '1.1',
 0,
 0,
 0,
 '29.7',
 '0.0',
 '0.1',
 0,
 0,
 '4.0',
 0,
 0,
 0,
 0,
 '33.7',
 '5.7',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '1.1',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '13.2',
 0,
 '8.3',
 '6.9',
 0,
 '2.5',
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '7.3',
 '16.1',
 '0.1',
 '5.3',
 0,
 0,
 '25.3',
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',
 0,
 0,
 '0.0',
 '37.1',
 '8.4',
 '1.1',
 0,
 0,
 0,
 0,
 0,
 0,
 '28.4',
 0,
 0,
 0,
 '1.1',
 0,
 0,
 0,
 0,
 0,
 '28.0',
 '45.7',
 0,
 0,
 0,
 '0.0',
 0,
 0,
 '9.2',
 '1.0',
 0,
 0,
 '0.2',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '112.5',
 '36.5',
 0,
 '91.0',
 '0.2',
 0,
 '0.0',
 0,
 0,
 0,
 0,
 0,
 0,
 '0.0',


In [162]:
date=crawling()
import pandas as pd

일조시간_2015=make_list()
cnt = 0
for num in date:

    일조시간_2015[cnt%13].append(num)    
    cnt+=1
    
일조시간_2015=일조시간_2015[1:]

일조시간_2015 = pd.DataFrame(일조시간_2015).T
일조시간_2015.drop(31,axis=0,inplace=True)
일조시간_2015_final=[]

for i in range(12):
    for j in range(31):
        일조시간_2015_final.append(일조시간_2015.iloc[j,i])
일조시간_2015_final

['9.2',
 '9.2',
 '9.4',
 '8.8',
 '5.1',
 '8.6',
 '9.1',
 '9.4',
 '9.3',
 '9.4',
 '9.1',
 '9.2',
 '6.9',
 '0.0',
 '3.7',
 '7.7',
 '9.5',
 '9.7',
 '9.0',
 '9.0',
 '0.5',
 '0.3',
 '9.7',
 '9.6',
 '3.5',
 '3.1',
 '5.3',
 '6.6',
 '0.0',
 '6.4',
 '10.1',
 '10.1',
 '7.7',
 '7.0',
 '4.0',
 '9.6',
 '9.9',
 '6.7',
 '10.0',
 '10.4',
 '9.9',
 '8.3',
 '9.6',
 '9.3',
 '10.3',
 '0.1',
 '0.0',
 '9.4',
 '9.6',
 '7.5',
 '9.8',
 '0.0',
 '3.9',
 '6.1',
 '7.5',
 '3.1',
 '7.2',
 '4.8',
 '0.0',
 '\xa0',
 '\xa0',
 '\xa0',
 '7.7',
 '11.0',
 '0.0',
 '9.6',
 '8.6',
 '10.4',
 '10.7',
 '10.8',
 '8.7',
 '11.3',
 '11.0',
 '11.3',
 '5.4',
 '10.7',
 '9.4',
 '9.9',
 '9.2',
 '0.0',
 '6.2',
 '5.8',
 '10.4',
 '10.5',
 '11.8',
 '11.5',
 '11.6',
 '11.8',
 '9.1',
 '8.0',
 '10.5',
 '11.1',
 '0.3',
 '0.0',
 '0.5',
 '4.9',
 '6.4',
 '0.0',
 '0.0',
 '1.3',
 '2.5',
 '11.3',
 '9.0',
 '12.3',
 '4.6',
 '0.1',
 '11.9',
 '10.8',
 '5.6',
 '11.4',
 '6.9',
 '0.0',
 '0.0',
 '11.5',
 '11.6',
 '10.6',
 '10.8',
 '12.5',
 '12.6',
 '12.4',
 '4.

In [125]:
date=crawling()
import pandas as pd

일조시간_2016=make_list()
cnt = 0
for num in date:

    일조시간_2016[cnt%13].append(num)    
    cnt+=1
    
일조시간_2016=일조시간_2016[1:]

일조시간_2016 = pd.DataFrame(일조시간_2016).T
일조시간_2016.drop(31,axis=0,inplace=True)
일조시간_2016_final=[]

for i in range(12):
    for j in range(31):

        일조시간_2016_final.append(일조시간_2016.iloc[j,i])
일조시간_2016_final

['9.3',
 '5.1',
 '9.2',
 '8.6',
 '0.0',
 '7.7',
 '8.6',
 '9.1',
 '9.4',
 '8.8',
 '7.4',
 '9.1',
 '9.3',
 '9.4',
 '8.5',
 '1.9',
 '0.0',
 '9.2',
 '9.5',
 '7.1',
 '2.0',
 '8.1',
 '5.2',
 '9.7',
 '9.8',
 '9.8',
 '9.4',
 '0.0',
 '0.0',
 '0.4',
 '7.5',
 '7.8',
 '8.8',
 '10.1',
 '7.5',
 '9.6',
 '10.1',
 '9.0',
 '10.2',
 '10.2',
 '10.4',
 '8.9',
 '0.0',
 '0.0',
 '6.1',
 '10.5',
 '9.7',
 '10.6',
 '10.6',
 '7.2',
 '2.6',
 '10.5',
 '1.9',
 '7.5',
 '7.3',
 '10.8',
 '8.2',
 '9.4',
 '6.5',
 '9.5',
 '\xa0',
 '\xa0',
 '10.9',
 '10.9',
 '10.8',
 '0.0',
 '0.1',
 '1.5',
 '8.8',
 '0.2',
 '5.8',
 '7.9',
 '11.5',
 '11.1',
 '0.0',
 '6.7',
 '10.6',
 '10.3',
 '9.9',
 '0.0',
 '9.9',
 '5.3',
 '11.2',
 '11.5',
 '7.5',
 '7.9',
 '3.7',
 '3.5',
 '11.0',
 '11.6',
 '4.9',
 '9.3',
 '5.4',
 '11.4',
 '7.1',
 '0.0',
 '0.0',
 '9.1',
 '5.0',
 '1.1',
 '8.8',
 '11.1',
 '8.9',
 '4.1',
 '7.3',
 '0.0',
 '11.5',
 '12.0',
 '3.0',
 '12.3',
 '7.7',
 '12.4',
 '10.5',
 '1.8',
 '11.9',
 '0.3',
 '9.0',
 '12.2',
 '10.9',
 '0.0',
 '1.7',

In [126]:
date=crawling()
import pandas as pd

일조시간_2017=make_list()
cnt = 0
for num in date:

    일조시간_2017[cnt%13].append(num)    
    cnt+=1
    
일조시간_2017=일조시간_2017[1:]

일조시간_2017 = pd.DataFrame(일조시간_2017).T
일조시간_2017.drop(31,axis=0,inplace=True)
일조시간_2017_final=[]

for i in range(12):
    for j in range(31):
        일조시간_2017_final.append(일조시간_2017.iloc[j,i])
일조시간_2017_final

['9.2',
 '8.9',
 '9.1',
 '8.0',
 '2.6',
 '6.8',
 '2.9',
 '1.9',
 '8.9',
 '9.4',
 '9.4',
 '7.3',
 '7.5',
 '9.5',
 '9.7',
 '9.7',
 '6.0',
 '5.7',
 '8.6',
 '8.7',
 '8.5',
 '9.4',
 '9.8',
 '10.0',
 '9.9',
 '9.8',
 '6.4',
 '9.6',
 '0.0',
 '9.4',
 '9.2',
 '5.5',
 '10.3',
 '10.1',
 '6.9',
 '3.2',
 '10.0',
 '6.2',
 '0.2',
 '9.9',
 '10.3',
 '10.4',
 '10.5',
 '10.1',
 '10.2',
 '10.4',
 '9.1',
 '9.5',
 '10.1',
 '9.9',
 '6.8',
 '10.1',
 '0.0',
 '9.2',
 '10.0',
 '10.4',
 '8.2',
 '7.9',
 '10.7',
 '\xa0',
 '\xa0',
 '\xa0',
 '7.7',
 '8.2',
 '10.8',
 '9.5',
 '3.2',
 '8.8',
 '11.1',
 '1.8',
 '\xa0',
 '\xa0',
 '\xa0',
 '\xa0',
 '3.4',
 '7.4',
 '11.3',
 '6.3',
 '11.0',
 '5.3',
 '10.5',
 '0.2',
 '6.4',
 '6.6',
 '5.0',
 '4.7',
 '4.6',
 '1.0',
 '8.8',
 '10.0',
 '0.0',
 '11.2',
 '0.1',
 '3.2',
 '8.2',
 '7.5',
 '11.4',
 '0.0',
 '0.0',
 '10.2',
 '11.0',
 '7.3',
 '4.8',
 '4.9',
 '11.3',
 '11.5',
 '9.1',
 '10.7',
 '10.8',
 '0.0',
 '9.7',
 '12.2',
 '3.4',
 '4.5',
 '7.2',
 '10.0',
 '10.6',
 '0.6',
 '2.3',
 '7.4',
 

In [129]:
date=crawling()
import pandas as pd

일조시간_2018=make_list()
cnt = 0
for num in date:

    일조시간_2018[cnt%13].append(num)    
    cnt+=1
    
일조시간_2018=일조시간_2018[1:]

일조시간_2018 = pd.DataFrame(일조시간_2018).T
일조시간_2018.drop(31,axis=0,inplace=True)
일조시간_2018_final=[]

for i in range(12):
    for j in range(31):
        일조시간_2018_final.append(일조시간_2018.iloc[j,i])
일조시간_2018_final

['9.3',
 '8.4',
 '9.0',
 '5.1',
 '6.3',
 '9.3',
 '1.6',
 '0.0',
 '8.9',
 '7.5',
 '9.5',
 '9.5',
 '8.2',
 '5.7',
 '9.3',
 '0.0',
 '1.1',
 '8.4',
 '8.9',
 '9.2',
 '7.8',
 '2.9',
 '9.5',
 '9.8',
 '9.3',
 '9.9',
 '9.9',
 '7.6',
 '9.7',
 '9.0',
 '6.8',
 '9.7',
 '6.8',
 '9.8',
 '10.1',
 '10.2',
 '10.3',
 '10.2',
 '10.0',
 '6.1',
 '4.4',
 '9.3',
 '9.7',
 '10.2',
 '8.7',
 '6.1',
 '4.2',
 '10.1',
 '6.3',
 '5.0',
 '9.3',
 '6.8',
 '10.5',
 '9.4',
 '9.4',
 '3.8',
 '10.5',
 '9.8',
 '0.2',
 '\xa0',
 '\xa0',
 '\xa0',
 '8.7',
 '10.4',
 '5.2',
 '0.9',
 '0.0',
 '9.1',
 '0.2',
 '0.0',
 '1.6',
 '8.6',
 '11.1',
 '10.7',
 '11.0',
 '10.5',
 '0.0',
 '6.4',
 '9.8',
 '4.5',
 '0.0',
 '1.9',
 '0.0',
 '7.8',
 '11.4',
 '9.8',
 '10.4',
 '11.0',
 '11.0',
 '11.0',
 '9.8',
 '11.1',
 '10.5',
 '7.0',
 '11.0',
 '10.9',
 '0.0',
 '0.0',
 '5.6',
 '10.7',
 '10.5',
 '10.7',
 '9.0',
 '10.0',
 '6.8',
 '3.8',
 '0.0',
 '11.1',
 '10.1',
 '7.7',
 '12.1',
 '12.2',
 '12.2',
 '11.6',
 '3.2',
 '0.0',
 '0.0',
 '12.3',
 '12.0',
 '10.6',
 

In [130]:
date=crawling()
import pandas as pd

일조시간_2019=make_list()
cnt = 0
for num in date:

    일조시간_2019[cnt%13].append(num)    
    cnt+=1
    
일조시간_2019=일조시간_2019[1:]

일조시간_2019 = pd.DataFrame(일조시간_2019).T
일조시간_2019.drop(31,axis=0,inplace=True)
일조시간_2019_final=[]

for i in range(12):
    for j in range(31):
        일조시간_2019_final.append(일조시간_2019.iloc[j,i])
일조시간_2019_final

['9.2',
 '9.3',
 '9.4',
 '4.8',
 '6.4',
 '7.1',
 '9.3',
 '8.8',
 '6.6',
 '6.6',
 '7.9',
 '0.2',
 '6.2',
 '8.7',
 '8.4',
 '9.5',
 '9.6',
 '9.7',
 '4.6',
 '9.4',
 '9.8',
 '9.7',
 '9.3',
 '9.5',
 '3.9',
 '9.8',
 '10.0',
 '8.4',
 '9.7',
 '7.8',
 '0.8',
 '10.0',
 '9.9',
 '0.0',
 '9.7',
 '9.7',
 '9.5',
 '2.6',
 '7.6',
 '8.5',
 '5.6',
 '10.1',
 '9.3',
 '9.8',
 '7.4',
 '0.4',
 '9.4',
 '10.5',
 '1.5',
 '0.4',
 '10.1',
 '9.1',
 '5.4',
 '8.1',
 '10.5',
 '10.1',
 '9.9',
 '0.6',
 '6.2',
 '\xa0',
 '\xa0',
 '\xa0',
 '9.7',
 '5.6',
 '2.1',
 '8.9',
 '6.8',
 '2.6',
 '6.7',
 '10.7',
 '10.2',
 '0.0',
 '10.1',
 '10.4',
 '10.9',
 '11.2',
 '5.8',
 '11.3',
 '11.4',
 '6.4',
 '10.7',
 '2.0',
 '8.3',
 '9.9',
 '9.3',
 '11.7',
 '4.7',
 '11.2',
 '10.3',
 '7.4',
 '9.2',
 '9.1',
 '11.4',
 '10.0',
 '11.8',
 '11.9',
 '11.5',
 '11.1',
 '11.3',
 '0.0',
 '11.0',
 '6.0',
 '0.3',
 '4.9',
 '10.1',
 '9.6',
 '2.7',
 '12.1',
 '11.0',
 '8.5',
 '11.5',
 '1.8',
 '12.0',
 '7.1',
 '11.8',
 '0.1',
 '1.8',
 '0.0',
 '0.0',
 '10.7',
 '4

In [132]:
date=crawling()
import pandas as pd


운량_2015=make_list()
cnt = 0
for num in date:

    운량_2015[cnt%13].append(num)    
    cnt+=1
    
운량_2015=운량_2015[1:]

운량_2015 = pd.DataFrame(운량_2015).T
운량_2015.drop(31,axis=0,inplace=True)
운량_2015_final=[]

for i in range(12):
    for j in range(31):
        운량_2015_final.append(운량_2015.iloc[j,i])
운량_2015_final

['0.4',
 '0.1',
 '2.8',
 '1.1',
 '6.5',
 '3.1',
 '2.5',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.3',
 '4.6',
 '10.0',
 '5.4',
 '3.9',
 '0.6',
 '2.3',
 '1.0',
 '3.8',
 '10.0',
 '9.3',
 '0.6',
 '0.0',
 '7.6',
 '7.4',
 '4.3',
 '3.1',
 '8.6',
 '4.0',
 '0.0',
 '0.0',
 '2.4',
 '4.4',
 '6.5',
 '1.6',
 '1.3',
 '5.9',
 '1.9',
 '0.0',
 '0.0',
 '2.4',
 '0.9',
 '0.1',
 '0.0',
 '8.3',
 '10.0',
 '1.5',
 '1.4',
 '1.8',
 '2.5',
 '9.8',
 '6.8',
 '4.6',
 '3.6',
 '8.3',
 '3.8',
 '4.6',
 '8.9',
 '\xa0',
 '\xa0',
 '\xa0',
 '5.1',
 '0.0',
 '7.6',
 '3.6',
 '4.0',
 '3.5',
 '2.3',
 '3.1',
 '4.3',
 '0.0',
 '0.1',
 '0.0',
 '6.0',
 '2.6',
 '4.5',
 '3.3',
 '3.4',
 '10.0',
 '6.3',
 '6.1',
 '2.1',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '4.4',
 '4.4',
 '1.0',
 '0.0',
 '7.6',
 '9.6',
 '9.4',
 '9.4',
 '7.8',
 '10.0',
 '10.0',
 '9.9',
 '7.8',
 '5.4',
 '2.6',
 '0.4',
 '8.1',
 '10.0',
 '4.9',
 '1.4',
 '5.0',
 '1.8',
 '6.5',
 '10.0',
 '10.0',
 '2.3',
 '3.1',
 '3.5',
 '5.8',
 '0.0',
 '0.0',
 '1.8',
 '8.4',
 '10.0',
 '7.8',
 '\

In [133]:
date=crawling()
import pandas as pd


운량_2016=make_list()
cnt = 0
for num in date:

    운량_2016[cnt%13].append(num)    
    cnt+=1
    
운량_2016=운량_2016[1:]

운량_2016 = pd.DataFrame(운량_2016).T
운량_2016.drop(31,axis=0,inplace=True)
운량_2016_final=[]

for i in range(12):
    for j in range(31):
        운량_2016_final.append(운량_2016.iloc[j,i])
운량_2016_final

['1.4',
 '4.6',
 '1.5',
 '0.8',
 '7.0',
 '5.6',
 '0.9',
 '0.8',
 '0.0',
 '0.4',
 '4.6',
 '1.1',
 '0.3',
 '1.4',
 '0.0',
 '6.4',
 '9.5',
 '2.9',
 '0.1',
 '2.8',
 '6.8',
 '4.8',
 '5.0',
 '1.9',
 '0.0',
 '0.0',
 '2.5',
 '10.0',
 '10.0',
 '6.8',
 '3.3',
 '2.9',
 '0.8',
 '0.0',
 '3.9',
 '0.6',
 '0.1',
 '4.0',
 '0.0',
 '0.0',
 '0.0',
 '5.5',
 '10.0',
 '9.3',
 '5.8',
 '1.0',
 '2.6',
 '0.0',
 '1.6',
 '6.5',
 '7.4',
 '1.0',
 '10.0',
 '6.5',
 '5.3',
 '0.0',
 '4.9',
 '2.6',
 '4.6',
 '2.6',
 '\xa0',
 '\xa0',
 '0.0',
 '0.0',
 '1.3',
 '10.0',
 '10.0',
 '8.6',
 '0.5',
 '9.8',
 '6.5',
 '4.8',
 '0.0',
 '2.1',
 '9.5',
 '2.3',
 '3.3',
 '4.1',
 '3.6',
 '9.5',
 '2.8',
 '7.9',
 '2.4',
 '2.9',
 '6.4',
 '6.6',
 '7.9',
 '6.0',
 '1.6',
 '0.0',
 '5.1',
 '4.4',
 '4.5',
 '0.0',
 '4.3',
 '9.8',
 '9.5',
 '3.6',
 '7.6',
 '7.9',
 '3.4',
 '4.0',
 '5.1',
 '7.0',
 '7.1',
 '9.5',
 '0.9',
 '0.4',
 '8.1',
 '3.1',
 '5.5',
 '0.0',
 '5.3',
 '7.8',
 '2.8',
 '9.5',
 '3.3',
 '2.6',
 '5.4',
 '10.0',
 '9.3',
 '2.4',
 '0.3',
 '\xa0'

In [134]:
date=crawling()
import pandas as pd


운량_2017=make_list()
cnt = 0
for num in date:

    운량_2017[cnt%13].append(num)    
    cnt+=1
    
운량_2017=운량_2017[1:]

운량_2017 = pd.DataFrame(운량_2017).T
운량_2017.drop(31,axis=0,inplace=True)
운량_2017_final=[]

for i in range(12):
    for j in range(31):
        운량_2017_final.append(운량_2017.iloc[j,i])
운량_2017_final

['1.9',
 '2.3',
 '0.0',
 '4.6',
 '8.0',
 '3.9',
 '6.3',
 '8.0',
 '1.4',
 '0.0',
 '2.1',
 '3.8',
 '2.0',
 '0.0',
 '0.0',
 '0.4',
 '4.0',
 '5.9',
 '2.4',
 '2.3',
 '2.8',
 '1.1',
 '0.0',
 '0.0',
 '0.0',
 '0.9',
 '4.4',
 '2.9',
 '10.0',
 '1.9',
 '1.4',
 '2.9',
 '0.0',
 '1.8',
 '7.1',
 '6.9',
 '0.1',
 '5.8',
 '9.5',
 '3.3',
 '0.0',
 '0.0',
 '0.0',
 '2.4',
 '1.6',
 '0.0',
 '6.6',
 '4.4',
 '2.3',
 '4.9',
 '4.4',
 '4.1',
 '9.9',
 '4.4',
 '1.0',
 '0.3',
 '4.9',
 '5.6',
 '1.5',
 '\xa0',
 '\xa0',
 '\xa0',
 '6.3',
 '3.9',
 '2.9',
 '2.6',
 '6.0',
 '2.6',
 '0.5',
 '0.4',
 '0.4',
 '0.0',
 '2.9',
 '5.1',
 '6.0',
 '6.3',
 '0.5',
 '6.4',
 '1.3',
 '4.1',
 '1.5',
 '7.4',
 '5.1',
 '6.1',
 '8.4',
 '9.0',
 '8.6',
 '9.6',
 '1.9',
 '4.5',
 '9.5',
 '3.8',
 '9.8',
 '8.8',
 '2.0',
 '0.0',
 '3.3',
 '9.9',
 '9.5',
 '4.4',
 '3.0',
 '6.5',
 '8.6',
 '6.5',
 '4.5',
 '0.0',
 '3.5',
 '4.1',
 '4.4',
 '9.0',
 '2.5',
 '2.0',
 '9.6',
 '7.5',
 '1.3',
 '0.0',
 '1.3',
 '9.4',
 '8.8',
 '4.6',
 '1.3',
 '0.0',
 '0.0',
 '\xa0',
 '1

In [135]:
date=crawling()
import pandas as pd


운량_2018=make_list()
cnt = 0
for num in date:

    운량_2018[cnt%13].append(num)    
    cnt+=1
    
운량_2018=운량_2018[1:]

운량_2018 = pd.DataFrame(운량_2018).T
운량_2018.drop(31,axis=0,inplace=True)
운량_2018_final=[]

for i in range(12):
    for j in range(31):
        운량_2018_final.append(운량_2018.iloc[j,i])
운량_2018_final

['0.0',
 '1.9',
 '3.6',
 '7.4',
 '3.8',
 '0.4',
 '7.8',
 '9.4',
 '3.4',
 '2.1',
 '0.0',
 '0.0',
 '0.6',
 '3.6',
 '2.1',
 '9.5',
 '6.3',
 '3.1',
 '0.9',
 '0.6',
 '5.8',
 '6.9',
 '1.3',
 '1.3',
 '5.5',
 '1.5',
 '2.5',
 '5.8',
 '1.0',
 '4.1',
 '7.1',
 '0.1',
 '6.5',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.5',
 '7.6',
 '7.5',
 '2.4',
 '1.3',
 '0.0',
 '4.5',
 '7.0',
 '6.1',
 '2.6',
 '7.1',
 '7.5',
 '4.1',
 '5.5',
 '2.5',
 '0.0',
 '5.3',
 '7.4',
 '0.0',
 '4.5',
 '9.8',
 '\xa0',
 '\xa0',
 '\xa0',
 '4.4',
 '2.4',
 '3.6',
 '8.3',
 '8.9',
 '5.1',
 '9.8',
 '9.0',
 '7.0',
 '4.1',
 '0.0',
 '0.0',
 '0.1',
 '2.8',
 '9.9',
 '6.4',
 '3.8',
 '7.8',
 '10.0',
 '9.9',
 '10.0',
 '3.5',
 '0.0',
 '1.5',
 '0.0',
 '1.0',
 '0.0',
 '0.3',
 '1.4',
 '0.0',
 '0.0',
 '5.5',
 '0.6',
 '2.8',
 '10.0',
 '10.0',
 '7.3',
 '4.5',
 '1.9',
 '4.0',
 '6.4',
 '6.1',
 '6.5',
 '8.0',
 '9.9',
 '3.8',
 '5.0',
 '4.6',
 '0.0',
 '0.0',
 '0.0',
 '2.9',
 '9.1',
 '10.0',
 '9.8',
 '0.4',
 '5.6',
 '2.9',
 '0.0',
 '2.8',
 '3.8',
 '\xa0',

In [136]:
date=crawling()
import pandas as pd


운량_2019=make_list()
cnt = 0
for num in date:

    운량_2019[cnt%13].append(num)    
    cnt+=1
    
운량_2019=운량_2019[1:]

운량_2019 = pd.DataFrame(운량_2019).T
운량_2019.drop(31,axis=0,inplace=True)
운량_2019_final=[]

for i in range(12):
    for j in range(31):
        운량_2019_final.append(운량_2019.iloc[j,i])
운량_2019_final[59]

['0.4',
 '0.0',
 '0.0',
 '5.4',
 '4.6',
 '4.3',
 '2.9',
 '3.6',
 '5.3',
 '4.8',
 '5.0',
 '8.9',
 '2.6',
 '3.0',
 '6.0',
 '1.5',
 '0.9',
 '2.1',
 '8.0',
 '2.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '4.8',
 '0.0',
 '0.3',
 '2.8',
 '0.8',
 '7.6',
 '7.0',
 '1.9',
 '2.5',
 '9.9',
 '4.1',
 '2.8',
 '5.4',
 '6.6',
 '5.4',
 '8.0',
 '7.0',
 '3.5',
 '3.3',
 '2.3',
 '5.9',
 '8.4',
 '5.9',
 '1.8',
 '9.3',
 '7.6',
 '2.6',
 '5.6',
 '7.9',
 '6.0',
 '1.6',
 '2.8',
 '4.3',
 '9.8',
 '6.0',
 '\xa0',
 '\xa0',
 '\xa0',
 '3.1',
 '5.1',
 '9.6',
 '4.8',
 '6.1',
 '7.4',
 '4.3',
 '0.6',
 '4.9',
 '9.1',
 '3.4',
 '2.0',
 '1.6',
 '1.5',
 '4.0',
 '0.0',
 '0.3',
 '8.3',
 '1.8',
 '9.0',
 '7.8',
 '3.0',
 '4.3',
 '0.0',
 '6.5',
 '0.5',
 '4.0',
 '6.9',
 '6.9',
 '5.9',
 '2.6',
 '2.4',
 '0.3',
 '0.0',
 '2.6',
 '2.1',
 '0.0',
 '7.4',
 '4.0',
 '7.4',
 '9.4',
 '6.6',
 '4.8',
 '5.3',
 '6.4',
 '0.0',
 '4.9',
 '4.8',
 '2.6',
 '7.8',
 '1.1',
 '6.8',
 '3.6',
 '9.4',
 '9.5',
 '9.9',
 '9.5',
 '4.8',
 '7.5',
 '9.9',
 '8.3',
 '\xa0',
 '7.

'\xa0'

In [163]:
'''
for i in [340,278,185,123,61,60,59]:
평균기온_2015[i]
평균기온_2015
평균기온_2015
평균기온_2015
평균기온_2015
강수량_2015
강수량_2015
강수량_2015
강수량_2015
강수량_2015
    del 일조시간_2015_final[i]
    del 일조시간_2016_final[i]
    del 일조시간_2017_final[i]
    del 일조시간_2018_final[i]
    del 일조시간_2019_final[i]
    del 운량_2015_final[i]
    del 운량_2016_final[i]
    del 운량_2017_final[i]
    del 운량_2018_final[i]
    del 운량_2019_final[i]
'''

In [169]:
df_2015 = pd.DataFrame([평균기온_2015_final,강수량_2015_final,일조시간_2015_final,운량_2015_final]).T
df_2016 = pd.DataFrame([평균기온_2016_final,강수량_2016_final,일조시간_2016_final,운량_2016_final]).T
df_2017 = pd.DataFrame([평균기온_2017_final,강수량_2017_final,일조시간_2017_final,운량_2017_final]).T
df_2018 = pd.DataFrame([평균기온_2018_final,강수량_2018_final,일조시간_2018_final,운량_2018_final]).T
df_2019 = pd.DataFrame([평균기온_2019_final,강수량_2019_final,일조시간_2019_final,운량_2019_final]).T
columns = ['평균기온','강수량','일조시간','운량']
df_2015.columns = columns
df_2016.columns = columns
df_2017.columns = columns
df_2018.columns = columns
df_2019.columns = columns

In [180]:
df_2016.tail(5)

,평균기온,강수량,일조시간,운량
360,4.3,2.6,8.7,3.1
361,3.7,0,9.0,1.4
362,2.5,0,9.0,1.4
363,2.4,0,9.0,1.0
364,4.7,0,7.6,2.3


In [358]:
df = df_2015.append(df_2016,ignore_index=True)
df = df.append(df_2017,ignore_index=True)
df = df.append(df_2018,ignore_index=True)
df = df.append(df_2019,ignore_index=True)
df

,평균기온,강수량,일조시간,운량
0,-2.2,0,9.2,0.4
1,-1.0,0,9.2,0.1
2,1.8,0,9.4,2.8
3,7.0,0,8.8,1.1
4,9.3,0.5,5.1,6.5
...,...,...,...,...
1820,3.6,0,9.2,0.0
1821,4.1,0,9.2,0.0
1822,6.5,11.4,0.0,8.6
1823,9.6,1.4,6.7,5.3


In [359]:
date = pd.DataFrame({'date':pd.date_range('2015-01-01', periods=365*5, freq='1D')})
date= date['date'].astype(str).tolist()
df['날짜']=date

In [361]:
df.columns=['평균기온','강수량','일조시간','운량','날짜']
df=df[['날짜','평균기온','강수량','일조시간','운량']]
df.to_csv('날짜데이터.csv')

In [2]:
import pandas as pd
df=pd.read_csv('날짜데이터.csv')
df=df[['날짜','평균기온','강수량','일조시간','운량']]

In [11]:
배추2015=pd.read_csv('./2015배추.csv',encoding='cp949')
배추2015=배추2015.drop('Unnamed: 1',axis=1).T
배추2015=배추2015.reset_index()
배추2015=배추2015.iloc[1:,:]
배추2015.columns=['날짜','배추가격평균']

배추2016=pd.read_csv('./2016배추2.csv',encoding='cp949')
배추2016=배추2016.drop('Unnamed: 1',axis=1).T
배추2016=배추2016.reset_index()
배추2016=배추2016.iloc[1:,:]
배추2016.columns=['날짜','배추가격평균']

배추2017=pd.read_csv('./2017배추.csv',encoding='cp949')
배추2017=배추2017.drop('Unnamed: 1',axis=1).T
배추2017=배추2017.reset_index()
배추2017=배추2017.iloc[1:,:]
배추2017.columns=['날짜','배추가격평균']

배추2018=pd.read_csv('./2018배추.csv',encoding='cp949')
배추2018=배추2018.drop('Unnamed: 1',axis=1).T
배추2018=배추2018.reset_index()
배추2018=배추2018.iloc[1:,:]
배추2018.columns=['날짜','배추가격평균']

배추2019=pd.read_csv('./2019배추.csv',encoding='cp949')
배추2019=배추2019.drop('Unnamed: 1',axis=1).T
배추2019=배추2019.reset_index()
배추2019=배추2019.iloc[1:,:]
배추2019.columns=['날짜','배추가격평균']


In [13]:
배추 = 배추2015.append(배추2016,ignore_index=False)
배추 = 배추.append(배추2017)
배추 = 배추.append(배추2018)
배추 = 배추.append(배추2019)
배추

,날짜,배추가격평균
1,2015-01-02,"4,000"
2,2015-01-05,"4,000"
3,2015-01-06,"4,000"
4,2015-01-07,"4,000"
5,2015-01-08,"4,000"
...,...,...
242,2019-12-24,"9,000"
243,2019-12-26,"9,000"
244,2019-12-27,"10,000"
245,2019-12-30,"10,000"


In [14]:
df['날짜']

0       2015-01-01
1       2015-01-02
2       2015-01-03
3       2015-01-04
4       2015-01-05
           ...    
1820    2019-12-26
1821    2019-12-27
1822    2019-12-28
1823    2019-12-29
1824    2019-12-30
Name: 날짜, Length: 1825, dtype: object

In [15]:
sample=pd.merge(left=df,
               right=배추,
               how='inner',
               left_on='날짜',
               right_on='날짜')

final_df= sample

In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226 entries, 0 to 1225
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      1226 non-null   object 
 1   평균기온    1226 non-null   float64
 2   강수량     1226 non-null   float64
 3   일조시간    1226 non-null   float64
 4   운량      1226 non-null   float64
 5   배추가격평균  1226 non-null   object 
dtypes: float64(4), object(2)
memory usage: 67.0+ KB


In [17]:
final_df.to_csv('토이프로젝트용다하고지워도됨1.csv',index=False)

In [373]:
aaa=pd.read_csv('토이프로젝트용다하고지워도됨.csv',index_col=0)

In [374]:
aaa

,평균기온,강수량,일조시간,운량,배추가격평균
날짜,,,,,
2015-01-02,-1.0,0.0,9.2,0.1,"4,000"
2015-01-05,9.3,0.5,5.1,6.5,"4,000"
2015-01-06,7.6,26.0,8.6,3.1,"4,000"
2015-01-07,1.4,0.0,9.1,2.5,"4,000"
2015-01-08,0.8,0.0,9.4,0.0,"4,000"
...,...,...,...,...,...
2019-12-23,7.7,0.0,7.9,3.0,"10,000"
2019-12-24,10.7,0.0,9.1,3.3,"9,000"
2019-12-26,3.6,0.0,9.2,0.0,"9,000"
